# The Sudoku Game

In [19]:
import numpy as np # import numpy package
import pyomo.environ as pyo # import pyomo package

In [20]:
# Data (known cell values)
known = [(1, 2, 4), (1, 4, 9), (1, 9, 7), (2, 1, 1), (2, 2, 9), (2, 4, 6), 
         (2, 9, 4), (3, 1, 5), (3, 8, 1), (4, 2, 8), (4, 5, 3), (4, 8, 7), 
         (5, 1, 2), (5, 6, 4), (5, 7, 5), (5, 9, 8), (6, 4, 5), (7, 8, 2), 
         (8, 7, 3), (8, 8, 4), (9, 2, 7), (9, 6, 6), (9, 9, 1)]

#This is data that is already given to the player at start.

In [21]:
# Cages (Blocks that add up to a sum)

cages = [[(1,1),(1,2),12]]

In [22]:
# Create a Concrete model
model = pyo.ConcreteModel()

#Abstract models are preferred for showing to people less familiar to coding, as it is more self-sufficient.

In [38]:
# Index sets
N = pyo.RangeSet(9)
B = pyo.RangeSet(3)
a = pyo.RangeSet(len(cages))

#Lists from 1-9 and 1-3 respectively

In [24]:
# Decision variables
model.x = pyo.Var(N,N,N, within=pyo.Binary)

#Since we have 9x9x9 variable options

In [25]:
# Objective function
model.obj = pyo.Objective(expr = 0)

In [26]:
# Uniqueness constraints
def unique_rule(model,i,j):
    return sum(model.x[i, j, k] for k in N) == 1

model.unique = pyo.Constraint(N,N, rule=unique_rule)

In [27]:
# Row constraints
def row_rule(model,i,k):
    return sum(model.x[i, j, k] for j in N) == 1

model.row = pyo.Constraint(N,N, rule=row_rule)

In [28]:
# Column constraints
def col_rule(model,j,k):
    return sum(model.x[i, j, k] for i in N) == 1

model.col = pyo.Constraint(N,N, rule=col_rule)

In [29]:
# Block constraints
def block_rule(model,s,t,k):
    return sum(model.x[i, j, k] for i in pyo.RangeSet(3*s-2,3*s) for j in pyo.RangeSet(3*t-2,3*t)) == 1

model.block = pyo.Constraint(B,B,N, rule=block_rule)

In [39]:
# Cage constraints
def cage_rule(model, k):
    for b in range(a):
        return sum(model.x[i,j,k] for i in cages[1][b] for j in cages[1][b]) == cages[1][b,-1]
model.cage = pyo.Constraint(N, N, N, a, cages, rule = cage_rule)

    'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


RuntimeError: Cannot add component 'cage_index' (type <class 'pyomo.core.base.set.SetProduct_OrderedSet'>) to block 'unknown': a component by that name (type <class 'pyomo.core.base.set.OrderedScalarSet'>) is already defined.

In [31]:
# Known cells constraints
def knownCell_rule(model,i,j,k):
    return model.x[i,j,k] == 1

model.knownCell = pyo.Constraint(known, rule=knownCell_rule)    

In [32]:
# Solve the model through the GLPK solver
solver = pyo.SolverFactory('glpk')
solver.solve(model)

    solver failure.


{'Problem': [{'Name': 'unknown', 'Lower bound': 0.0, 'Upper bound': 0.0, 'Number of objectives': 1, 'Number of constraints': 348, 'Number of variables': 730, 'Number of nonzeros': 2940, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.07083630561828613}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [33]:
# In case you are interested in the solution
model.display()

Model unknown

  Variables:
    x : Size=729, Index=x_index
        Key       : Lower : Value : Upper : Fixed : Stale : Domain
        (1, 1, 1) :     0 :   0.0 :     1 : False : False : Binary
        (1, 1, 2) :     0 :   0.0 :     1 : False : False : Binary
        (1, 1, 3) :     0 :   1.0 :     1 : False : False : Binary
        (1, 1, 4) :     0 :   0.0 :     1 : False : False : Binary
        (1, 1, 5) :     0 :   0.0 :     1 : False : False : Binary
        (1, 1, 6) :     0 :   0.0 :     1 : False : False : Binary
        (1, 1, 7) :     0 :   0.0 :     1 : False : False : Binary
        (1, 1, 8) :     0 :   0.0 :     1 : False : False : Binary
        (1, 1, 9) :     0 :   0.0 :     1 : False : False : Binary
        (1, 2, 1) :     0 :   0.0 :     1 : False : False : Binary
        (1, 2, 2) :     0 :   0.0 :     1 : False : False : Binary
        (1, 2, 3) :     0 :   0.0 :     1 : False : False : Binary
        (1, 2, 4) :     0 :   1.0 :     1 : False : False : Binary
  

In [34]:
# Print the solution
sol_array = np.zeros((9,9))

for i in N:
    for j in N:
        for k in N:
            if pyo.value(model.x[i,j,k]) == 1:
                sol_array[i-1][j-1] = k

print(sol_array)

[[3. 4. 2. 9. 1. 5. 6. 8. 7.]
 [1. 9. 7. 6. 8. 3. 2. 5. 4.]
 [5. 6. 8. 4. 2. 7. 9. 1. 3.]
 [6. 8. 5. 1. 3. 2. 4. 7. 9.]
 [2. 1. 9. 7. 6. 4. 5. 3. 8.]
 [7. 3. 4. 5. 9. 8. 1. 6. 2.]
 [8. 5. 1. 3. 4. 9. 7. 2. 6.]
 [9. 2. 6. 8. 7. 1. 3. 4. 5.]
 [4. 7. 3. 2. 5. 6. 8. 9. 1.]]
